In [1]:
%load_ext pycodestyle_magic

In [2]:
%flake8_on

In [3]:
testdata = """1 + 2 * 3 + 4 * 5 + 6
2 * 3 + (4 * 5)
5 + (8 * 3 + 9 + 3 * 4 * 3)
5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))
((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2""".splitlines()

with open('input', 'r') as inp:
    inputdata = [line.strip() for line in inp.readlines()]

In [4]:
def shunting_yard(expr_str, silent=False, part2=False):
    output = []
    operators = []
    # while there are tokens to be read:
    for token in expr_str.split():
        if not silent:
            print(f'read a token: {token}')
        if token in {'+', '*'}:
            if not silent:
                print(f'if token is an operator then: {token}')
            while (len(operators) > 0 and operators[-1] != '('
                   and (not part2 or
                        (not token == '+' and not operators[-1] == '*'))
                   ):
                if not silent:
                    print('while there is an operator at the top of the'
                          + ' operator stack (not a left parenthesis): '
                          + operators[-1])
                    # and ((the operator at the top of the operator stack has
                    #  greater precedence) or (the operator at the top of the
                    #  operator stack has equal precedence and the token is
                    #  left associative))
                if not silent:
                    print('pop operators from the operator stack onto the'
                          + f' output queue: {operators[-1]}')
                output.append(operators.pop())
            if not silent:
                print(f'push it onto the operator stack: {token}')
            operators.append(token)
        elif token[0] == '(':
            if not silent:
                print('else if the token is a left parenthesis then: '
                      + token[0])
            while token[0] == '(':
                if not silent:
                    print(f'push it onto the operator stack: {token[0]}')
                operators.append(token[0])
                token = token[1:]
            if not silent:
                print(f'push it to the output queue: {int(token)}')
            output.append(int(token))
        elif token[-1] == ')':
            rp_count = 0
            while token[-1] == ')':
                if not silent:
                    print('else if the token is a right parenthesis: '
                          + token[-1])
                rp_count += 1
                token = token[:-1]
            if not silent:
                print(f'push it to the output queue: {int(token)}')
            output.append(int(token))
            while rp_count > 0:
                rp_count -= 1
                # while the operator at the top of the operator stack is not
                # a left parenthesis:
                while operators[-1] != '(':
                    if not silent:
                        print('pop operators from the operator stack onto'
                              + ' the output queue: {operators[-1]}')
                    output.append(operators.pop())
                    # /* If the stack runs out without finding a left
                    # parenthesis, then there are mismatched parentheses. */
                if not silent:
                    print('if there is a left parenthesis at the top of the'
                          + 'operator stack, then: pop the operator from the'
                          + f'operator stack and discard it: {operators[-1]}')
                operators.pop()
        # else if the token is a number, then:
        else:
            if not silent:
                print(f'push it to the output queue: {int(token)}')
            output.append(int(token))
    # /* After while loop, if operator stack not null,
    # pop everything to output queue */
    # if there are no more tokens to read then:
    #     while there are still operator tokens on the stack:
    while len(operators) > 0:
        # /* If the operator token on the top of the stack is a parenthesis,
        # then there are mismatched parentheses. */
        if not silent:
            print('pop the operator from the operator stack onto the'
                  + f' output queue: {operators[-1]}')
        output.append(operators.pop())
    # exit.
    return output

In [5]:
def evaluate(expr_rpn):
    stack = []
    for token in expr_rpn:
        if token == '+':
            stack.append(stack.pop() + stack.pop())
        elif token == '*':
            stack.append(stack.pop() * stack.pop())
        else:
            stack.append(token)
    return stack.pop()

In [6]:
print(testdata[0])
expr_rpn = shunting_yard(testdata[0])
print(expr_rpn)
evaluate(expr_rpn)

1 + 2 * 3 + 4 * 5 + 6
read a token: 1
push it to the output queue: 1
read a token: +
if token is an operator then: +
push it onto the operator stack: +
read a token: 2
push it to the output queue: 2
read a token: *
if token is an operator then: *
while there is an operator at the top of the operator stack (not a left parenthesis): +
pop operators from the operator stack onto the output queue: +
push it onto the operator stack: *
read a token: 3
push it to the output queue: 3
read a token: +
if token is an operator then: +
while there is an operator at the top of the operator stack (not a left parenthesis): *
pop operators from the operator stack onto the output queue: *
push it onto the operator stack: +
read a token: 4
push it to the output queue: 4
read a token: *
if token is an operator then: *
while there is an operator at the top of the operator stack (not a left parenthesis): +
pop operators from the operator stack onto the output queue: +
push it onto the operator stack: *
read 

71

In [7]:
for expr_str in testdata:
    expr_rpn = shunting_yard(expr_str, silent=True)
    result = evaluate(expr_rpn)
    print(expr_str, expr_rpn, result)

1 + 2 * 3 + 4 * 5 + 6 [1, 2, '+', 3, '*', 4, '+', 5, '*', 6, '+'] 71
2 * 3 + (4 * 5) [2, 3, '*', 4, 5, '*', '+'] 26
5 + (8 * 3 + 9 + 3 * 4 * 3) [5, 8, 3, '*', 9, '+', 3, '+', 4, '*', 3, '*', '+'] 437
5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) [5, 9, '*', 7, 3, '*', 3, '*', 9, '+', 3, '*', 8, 6, '+', 4, '*', '+', '*'] 12240
((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 [2, 4, '+', 9, '*', 6, 9, '+', 8, '*', 6, '+', '*', 6, '+', 2, '+', 4, '+', 2, '*'] 13632


In [8]:
cumsum = 0
for expr_str in inputdata:
    expr_rpn = shunting_yard(expr_str, silent=True)
    result = evaluate(expr_rpn)
    print(expr_str, expr_rpn, result)
    cumsum += result
print(cumsum)

, 6, '+', 4, '*', 5, '+', 6, '*', 8, '+'] 59822
3 + 8 * 9 + 3 + 5 [3, 8, '+', 9, '*', 3, '+', 5, '+'] 107
(6 + (3 * 9 * 3 + 2 * 7 * 9) * (5 + 7 + 2 * 7 * 8)) + 2 + 8 * 2 + 7 + 2 [6, 3, 9, '*', 3, '*', 2, '+', 7, '*', 9, '*', '+', 5, 7, '+', 2, '+', 7, '*', 8, '*', '*', 2, '+', 8, '+', 2, '*', 7, '+', 2, '+'] 8208509
6 * (5 * 9 + 8 + 5) * (5 + 9 * 7 + 7 * 9 * (2 + 6 * 2)) * 5 [6, 5, 9, '*', 8, '+', 5, '+', '*', 5, 9, '+', 7, '*', 7, '+', 9, '*', 2, 6, '+', 2, '*', '*', '*', 5, '*'] 26308800
6 * ((6 * 3 + 6 * 2 + 8) + 8 + (3 + 2 * 5 * 7 * 4) + (8 + 7) + 4) [6, 6, 3, '*', 6, '+', 2, '*', 8, '+', 8, '+', 3, 2, '+', 5, '*', 7, '*', 4, '*', '+', 8, 7, '+', '+', 4, '+', '*'] 4698
5 + ((2 * 4 + 4) + 9) + (2 * 6 + 5 + 8 * 2) + 5 * 4 * 6 [5, 2, 4, '*', 4, '+', 9, '+', '+', 2, 6, '*', 5, '+', 8, '+', 2, '*', '+', 5, '+', 4, '*', 6, '*'] 1944
(8 * 6 * 6 + 4) * 7 + 5 * (7 + 8 + (4 * 6 + 7 * 3 * 6 * 7) * (9 + 6) * 7) + 7 [8, 6, '*', 6, '*', 4, '+', 7, '*', 5, '+', 7, 8, '+', 4, 6, '*', 7, '+', 3, '*

In [9]:
print(testdata[0])
expr_rpn = shunting_yard(testdata[0], part2=True)
print(expr_rpn)
evaluate(expr_rpn)

1 + 2 * 3 + 4 * 5 + 6
read a token: 1
push it to the output queue: 1
read a token: +
if token is an operator then: +
push it onto the operator stack: +
read a token: 2
push it to the output queue: 2
read a token: *
if token is an operator then: *
while there is an operator at the top of the operator stack (not a left parenthesis): +
pop operators from the operator stack onto the output queue: +
push it onto the operator stack: *
read a token: 3
push it to the output queue: 3
read a token: +
if token is an operator then: +
push it onto the operator stack: +
read a token: 4
push it to the output queue: 4
read a token: *
if token is an operator then: *
while there is an operator at the top of the operator stack (not a left parenthesis): +
pop operators from the operator stack onto the output queue: +
push it onto the operator stack: *
read a token: 5
push it to the output queue: 5
read a token: +
if token is an operator then: +
push it onto the operator stack: +
read a token: 6
push it to

231

In [10]:
for expr_str in testdata:
    expr_rpn = shunting_yard(expr_str, silent=True, part2=True)
    result = evaluate(expr_rpn)
    print(expr_str, expr_rpn, result)

1 + 2 * 3 + 4 * 5 + 6 [1, 2, '+', 3, 4, '+', 5, 6, '+', '*', '*'] 231
2 * 3 + (4 * 5) [2, 3, 4, 5, '*', '+', '*'] 46
5 + (8 * 3 + 9 + 3 * 4 * 3) [5, 8, 3, 9, 3, '+', '+', 4, 3, '*', '*', '*', '+'] 1445
5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) [5, 9, 7, 3, 3, 9, '+', 3, 8, 6, '+', 4, '*', '+', '*', '*', '*', '*', '*'] 669060
((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 [2, 4, '+', 9, '*', 6, 9, '+', 8, 6, '+', '*', 6, '+', '*', 2, 4, '+', '+', 2, '*'] 23340


In [11]:
cumsum = 0
for expr_str in inputdata:
    expr_rpn = shunting_yard(expr_str, silent=True, part2=True)
    result = evaluate(expr_rpn)
    print(expr_str, expr_rpn, result)
    cumsum += result
print(cumsum)

6 + (3 * 9 * 3 + 2 * 7 * 9) * (5 + 7 + 2 * 7 * 8)) + 2 + 8 * 2 + 7 + 2 [6, 3, 9, 3, 2, '+', 7, 9, '*', '*', '*', '*', '+', 5, 7, 2, '+', '+', 7, 8, '*', '*', '*', 2, 8, '+', '+', 2, 7, 2, '+', '+', '*'] 73398974
6 * (5 * 9 + 8 + 5) * (5 + 9 * 7 + 7 * 9 * (2 + 6 * 2)) * 5 [6, 5, 9, 8, 5, '+', '+', '*', 5, 9, '+', 7, 7, '+', 9, 2, 6, '+', 2, '*', '*', '*', '*', 5, '*', '*', '*'] 93139200
6 * ((6 * 3 + 6 * 2 + 8) + 8 + (3 + 2 * 5 * 7 * 4) + (8 + 7) + 4) [6, 6, 3, 6, '+', 2, 8, '+', '*', '*', 8, 3, 2, '+', 5, 7, 4, '*', '*', '*', 8, 7, '+', 4, '+', '+', '+', '+', '*'] 7602
5 + ((2 * 4 + 4) + 9) + (2 * 6 + 5 + 8 * 2) + 5 * 4 * 6 [5, 2, 4, 4, '+', '*', 9, '+', 2, 6, 5, 8, '+', '+', 2, '*', '*', 5, '+', '+', '+', 4, 6, '*', '*'] 2664
(8 * 6 * 6 + 4) * 7 + 5 * (7 + 8 + (4 * 6 + 7 * 3 * 6 * 7) * (9 + 6) * 7) + 7 [8, 6, 6, 4, '+', '*', '*', 7, 5, '+', 7, 8, 4, 6, 7, '+', 3, 6, 7, '*', '*', '*', '*', '+', '+', 9, 6, '+', 7, '*', '*', 7, '+', '*', '*'] 3971761920
3 * (4 * 3 * 9 + 3 * 6) * (6 * 5) 